In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap

In [ ]:
# Import Data
yamal = ee.ImageCollection('projects/abruptthawmapping/assets/planet_yamal_calibrated_initial_data')
polygons = ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_for_Yili_Apr_2022').filterMetadata('code_id', 'equals', 1)

# Prep Map
Map = geemap.Map()
Map.centerObject(yamal)

In [ ]:
# View the imagery
vis_params_imagery = {
    'min': [100, 100, 200],'max': [1000, 1100, 580],
    'bands': ['b3', 'b2', 'b1'],
    'gamma': 0.9
}

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return(image.updateMask(mask))

final = yamal.map(mask_0)

In [ ]:
# Set Properties to allow filtering
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;


yamal = yamal.map(setID);

In [ ]:
# Filter by year
yamal_2017 = yamal.filter(ee.Filter.stringContains('ID', '2017'))
yamal_2018 = yamal.filter(ee.Filter.stringContains('ID', '2018'))
yamal_2019 = yamal.filter(ee.Filter.stringContains('ID', '2019'))
yamal_2020 = yamal.filter(ee.Filter.stringContains('ID', '2020'))
yamal_2021 = yamal.filter(ee.Filter.stringContains('ID', '2021'))

In [ ]:
# Median Composite 2017
final_2017 = yamal_2017.select(['b1', 'b2', 'b3', 'b4']).median()
final_2018 = yamal_2018.select(['b1', 'b2', 'b3', 'b4']).median()
final_2019 = yamal_2019.select(['b1', 'b2', 'b3', 'b4']).median()
final_2020 = yamal_2020.select(['b1', 'b2', 'b3', 'b4']).median()
final_2021 = yamal_2021.select(['b1', 'b2', 'b3', 'b4']).median()
final_all = yamal.select(['b1', 'b2', 'b3', 'b4']).median()
Map.addLayer(final_2017, vis_params_imagery, 'Median 2017')
Map.addLayer(final_2018, vis_params_imagery, 'Median 2018')
Map.addLayer(final_2019, vis_params_imagery, 'Median 2019')
Map.addLayer(final_2020, vis_params_imagery, 'Median 2020')
Map.addLayer(final_2021, vis_params_imagery, 'Median 2021')
Map.addLayer(final_all, vis_params_imagery, 'Median All Years')

In [ ]:
# 25th Percentile Composite (25th Percentile Darkest)
final_2017_25th = yamal_2017.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
final_2018_25th = yamal_2018.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
final_2019_25th = yamal_2019.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
final_2020_25th = yamal_2020.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
final_2021_25th = yamal_2021.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
final_all_25th = yamal.reduce(ee.Reducer.percentile([25])).select(['b1_p25', 'b2_p25', 'b3_p25', 'b4_p25'], ['b1', 'b2', 'b3', 'b4'])
Map.addLayer(final_2017_25th, vis_params_imagery, '25th Percentile 2017')
Map.addLayer(final_2018_25th, vis_params_imagery, '25th Percentile 2018')
Map.addLayer(final_2019_25th, vis_params_imagery, '25th Percentile 2019')
Map.addLayer(final_2020_25th, vis_params_imagery, '25th Percentile 2020')
Map.addLayer(final_2021_25th, vis_params_imagery, '25th Percentile 2021')
Map.addLayer(final_all_25th, vis_params_imagery, '25th All Years')

In [ ]:
Map.addLayer(polygons, {'color': 'FF0000'}, 'RTS');

In [ ]:
Map

In [ ]:
# Export Composite to Assets
roi = ee.FeatureCollection('users/hrodenhizer-woodwellclimate/Yamal_Aoi').first();
geometry = roi.geometry()
scale = yamal.first().select('b1').projection().nominalScale().getInfo()

In [ ]:
### Export Single Year, Median Mosaics
# Export 2017
task = ee.batch.Export.image.toAsset(
    image = final_2017,
    description = 'Planet_Median_Composite_Calibrated_2017',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_2017',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2018
task = ee.batch.Export.image.toAsset(
    image = final_2018,
    description = 'Planet_Median_Composite_Calibrated_2018',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_2018',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2019
task = ee.batch.Export.image.toAsset(
    image = final_2019,
    description = 'Planet_Median_Composite_Calibrated_2019',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_2019',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2020
task = ee.batch.Export.image.toAsset(
    image = final_2020,
    description = 'Planet_Median_Composite_Calibrated_2020',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_2020',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2021
task = ee.batch.Export.image.toAsset(
    image = final_2021,
    description = 'Planet_Median_Composite_Calibrated_2021',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_2021',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

## Export 5-year, Median Mosaics
task = ee.batch.Export.image.toAsset(
    image = final_all,
    description = 'Planet_Median_Composite_Calibrated_All',
    assetId = 'projects/abruptthawmapping/assets/planet_median_calibrated_all',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

In [ ]:
### Export Single Year, 25th Percentile Mosaics
# Export 2017
task = ee.batch.Export.image.toAsset(
    image = final_2017_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_2017',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_2017',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2018
task = ee.batch.Export.image.toAsset(
    image = final_2018_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_2018',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_2018',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2019
task = ee.batch.Export.image.toAsset(
    image = final_2019_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_2019',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_2019',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2020
task = ee.batch.Export.image.toAsset(
    image = final_2020_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_2020',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_2020',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

# Export 2021
task = ee.batch.Export.image.toAsset(
    image = final_2021_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_2021',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_2021',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()

## Export 5-year, 25th Percentile Mosaics
task = ee.batch.Export.image.toAsset(
    image = final_all_25th,
    description = 'Planet_25th_Percentile_Composite_Calibrated_All',
    assetId = 'projects/abruptthawmapping/assets/planet_25th_percentile_calibrated_all',
    crs = 'EPSG:32642',
    region = geometry,
    scale = scale,
    pyramidingPolicy = {
        '.default': 'mean'
    },
    maxPixels = 1e13
)
task.start()